In [9]:
import pandas as pd

df = pd.read_parquet("hf://datasets/Shiyu-Lab/Wikipedia_Person_Unlearn/forget_100_hard_retain/train-00000-of-00001.parquet")

In [14]:
df.head()

,Unnamed: 0,title,question,answer
0,0,Benedetto Varchi,In which Italian region is Montevarchi located?,Tuscany
1,1,Benedetto Varchi,Which Italian dialect served as the basis for ...,Florentine dialect
2,2,Benedetto Varchi,When did Ezra Pound begin writing The Cantos?,1915
3,3,Benedetto Varchi,How many sections are in The Cantos?,120
4,4,Benedetto Varchi,What was the name of the council that ruled th...,Signoria of Florence


In [12]:
df.to_csv("wikipedia_hard_retain_full.csv")

In [15]:
from sklearn.model_selection import train_test_split

# Load the dataset
file_path = "wikipedia_hard_retain_full.csv"  # Replace with the actual file path
df = pd.read_csv(file_path)

# Initialize empty DataFrames for training and testing sets
train_df = pd.DataFrame()
test_df = pd.DataFrame()

for title in df['title'].unique():
    title_data = df[df['title'] == title]
    
    if len(title_data) > 1:  # Only split if there are more than one sample
        train, test = train_test_split(title_data, test_size=0.2, random_state=42)
        train_df = pd.concat([train_df, train], ignore_index=True)
        test_df = pd.concat([test_df, test], ignore_index=True)
    else:
        # Assign small group to the training set
        train_df = pd.concat([train_df, title_data], ignore_index=True)

print("Training set size:", len(train_df))
print("Test set size:", len(test_df))

Training set size: 1449
Test set size: 377


In [17]:
train_df.head()

,Unnamed: 0,title,question,answer
0,8,Benedetto Varchi,What are the main subdivisions of Tuscan diale...,"Northern Tuscan dialects, Southern Tuscan dial..."
1,5,Benedetto Varchi,Who was the first member of the Medici family ...,Cosimo de' Medici
2,11,Benedetto Varchi,Who was Piero Strozzi's father?,Filippo Strozzi the Younger
3,3,Benedetto Varchi,How many sections are in The Cantos?,120
4,18,Benedetto Varchi,What historical event marked the beginning of ...,The town sprang up around 1100 near a fortifie...


In [18]:
test_df.head()

,Unnamed: 0,title,question,answer
0,0,Benedetto Varchi,In which Italian region is Montevarchi located?,Tuscany
1,17,Benedetto Varchi,What was Piero Strozzi's relation to the Medic...,He was the son of Clarice de' Medici and marri...
2,15,Benedetto Varchi,Name a dialect of Tuscan spoken in the city of...,Pisano-Livornese
3,1,Benedetto Varchi,Which Italian dialect served as the basis for ...,Florentine dialect
4,20,Wilhelm Wattenbach,Where has the Monumenta Germaniae Historica In...,Munich


In [19]:
train_df.shape

(1449, 4)

In [20]:
test_df.shape

(377, 4)

In [21]:
train_df.to_csv("train_wikipedia_hard_retain_100.csv", index=False)
test_df.to_csv("test_wikipedia_hard_retain_100.csv", index=False)

In [22]:
# Load the dataset
csv_file_path = 'train_wikipedia_hard_retain_100.csv'  # Replace with your dataset path
data = pd.read_csv(csv_file_path)

In [23]:
data

,Unnamed: 0,title,question,answer
0,8,Benedetto Varchi,What are the main subdivisions of Tuscan diale...,"Northern Tuscan dialects, Southern Tuscan dial..."
1,5,Benedetto Varchi,Who was the first member of the Medici family ...,Cosimo de' Medici
2,11,Benedetto Varchi,Who was Piero Strozzi's father?,Filippo Strozzi the Younger
3,3,Benedetto Varchi,How many sections are in The Cantos?,120
4,18,Benedetto Varchi,What historical event marked the beginning of ...,The town sprang up around 1100 near a fortifie...
...,...,...,...,...
1444,1818,Edward Adelbert Doisy,"When was the railroad through Edgar County, wh...",In the spring of 1873
1445,1813,Edward Adelbert Doisy,What are the fields in which the Nobel Prizes ...,"Physics, Chemistry, Physiology or Medicine, Li..."
1446,1816,Edward Adelbert Doisy,What is the purpose of the American Academy of...,To cultivate every art and science which may t...
1447,1820,Edward Adelbert Doisy,How is the Nobel Prize for Economic Sciences r...,It was endowed by Sweden's central bank in 196...


In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [26]:
model_name = "nvidia/Nemotron-Mini-4B-Instruct"  # Hugging Face model path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

In [28]:
# Define the question and correct answer columns
question_column = 'question'
correct_answer_column = 'answer'

In [ ]:
# Define the system and user prompt components
system_prompt = """
<extra_id_0>System
You are an AI tasked with generating a list of four unique incorrect answers for a given question. 
Your task is to:
1. Generate exactly four unique incorrect answers.
2. Ensure none of the incorrect answers match the provided correct answer.
3. Return the output strictly in the following format:
   {
       "question": "<question>",
       "correct_answer": "<correct_answer>",
       "wrong_answers": ["<wrong_answer_1>", "<wrong_answer_2>", "<wrong_answer_3>", "<wrong_answer_4>"]
   }
4. Do not include explanations, additional context, or any text outside the specified JSON format of the output.
"""

# Function to generate incorrect answers using Nemotron-Mini-4B-Instruct format
def generate_incorrect_answers(question, correct_answer):
    # Construct the user prompt
    user_prompt = f"<extra_id_1>User\n{{\"question\": \"{question}\", \"correct_answer\": \"{correct_answer}\"}}\n<extra_id_1>Assistant\n"
    
    # Combine system and user prompts
    full_prompt = f"{system_prompt}\n\n{user_prompt}"
    
    # Tokenize and generate response
    input_ids = tokenizer.encode(full_prompt, return_tensors="pt").to("cuda")
    output = model.generate(input_ids, max_length=300, temperature=0.4,pad_token_id = tokenizer.eos_token_id)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Extract the JSON object from the generated response
    start_idx = generated_text.find("{")
    end_idx = generated_text.rfind("}") + 1
    if start_idx != -1 and end_idx != -1:
        json_response = generated_text[start_idx:end_idx]
        # Clean up to ensure no additional text outside JSON
        return json_response.strip()
    else:
        return "Invalid response format"
sample_data = data[:3]
# Process each question in the dataset
results = []
for index, row in data.iterrows():
    print(index)
    question = row[question_column]
    correct_answer = row[correct_answer_column]
    response = generate_incorrect_answers(question, correct_answer)
    results.append({"Response": response})

# Save results to a CSV
output_df = pd.DataFrame(results)
output_df.to_csv("cleaned_incorrect_answers_wikipedia_hard_retain.csv", index=False)

print("Generated responses saved to 'cleaned_incorrect_answers_wikipedia_hard_retain.csv'.")

0


/home/akamani/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/akamani/.local/lib/python3.11/site-packages/transformers/models/nemotron/modeling_nemotron.py:133: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
